<a href="https://colab.research.google.com/github/raulpoppiel/sugarcane-yield-prediction/blob/main/06_merge_data_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><font color="green" size="6"> <b>Sugarcane Yield Prediction using RS and ML<b> </font></center>
<center><img src="https://www.omex.com/wp-content/uploads/2020/10/Sugar-Cane-Harvest-1536x772.jpg" height="150"></center>

---

_Script elaborated by **Raul Roberto Poppiel**_ ([raulpoppiel@gmail.com](raulpoppiel@gmail.com)) for the [FAPESP](https://fapesp.br/en) project Nº [23/01062-1](https://bv.fapesp.br/en/bolsas/207973/satellite-imagery-and-machine-learning-for-sugarcane-yield-estimation-in-regions-of-sao-paulo-state/), led by Professor Ana Claudia dos Santos Luciano (analuciano@usp.br) from ESALQ/USP, Brazil. The methodology employed was developed by Rafaella Pironato Amaro and is detailed in the document titled [Estimativa de produtividade da cana-de-açúcar a partir de imagens do satélite Sentinel-2A e o algoritmo de aprendizagem de máquina Random Forest](https://doi.org/10.11606/D.11.2023.tde-02102023-163947).

### Install and import tools

Data science modules

In [ ]:
# Load modules
import os
import pandas as pd
import geopandas as gpd
from pathlib import Path

print('Modules loaded')

Modules loaded


### Connect to GDrive

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set GDrive paths

Define the output folders in GD

In [ ]:
# Define folder names
folder_name_root = 'Colab Notebooks'
folder_name_project = '01_SugarcaneYieldPrediction'
folder_name_farm = 'usinas_all'
folder_name_specific = '06_merge_data_all'

# Define sugarcane crop-season (SAFRA)
CropSeason = 1920 # 1920, 2021, 2122, 2223

In [ ]:
# Check if the folder exists or else create
root_path = f'/content/drive/MyDrive/{folder_name_root}'
project_path = f'{root_path}/{folder_name_project}'
farm_path = f'{project_path}/{folder_name_farm}'
out_path = f'{farm_path}/{folder_name_specific}' # your results will be stored in 'out_path'

if not os.path.exists(out_path):
  Path(out_path).mkdir(parents=True, exist_ok=True)
  print("Output directory created successfully.")
else:
  print("Output directory already exists.")

os.chdir(out_path)
print(os.getcwd(),'\n')  # Print the current working directory
pd.DataFrame(os.listdir(), columns=['List files'])  # List files and directories in the current directory

Output directory already exists.
/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/06_merge_data_all 

List of files:
                                 Files
0  06_all_data_monthly_safra_1920.csv
1  06_all_data_monthly_safra_2021.csv
2  06_all_data_monthly_safra_2122.csv
3  06_all_data_monthly_safra_2223.csv


# Merge monthly Satellite-BH data

In [ ]:
# Import CSV of monthly data
monthly_CL = f'{farm_path}/03_climatic_data_monthly/02_climatic_data_monthlySource_MEAN_safra_{CropSeason}_merged.csv'
monthly_HB   = f'{farm_path}/04_hydricBalance_data_monthly/04_HydricBalance_data_monthly_safra_{CropSeason}.csv'
monthly_S2   = f'{farm_path}/05_sentinel2_data_monthly/01_sentinel2_data_monthly_MEAN_safra_{CropSeason}_merged.csv'

CL_monthly = pd.read_csv(monthly_CL)
HB_monthly = pd.read_csv(monthly_HB)
S2_monthly = pd.read_csv(monthly_S2)

Import shapefile with Agronomic data

In [ ]:
# Path
shp_path = f'{farm_path}/01_input_data/usinas_merge_Dez22_dissolve_joinedData_epsg4326.shp'

In [ ]:
# Read shapefile
gdf = gpd.read_file(shp_path)
print('Rows:', gdf.shape[0])

Rows: 25492


In [ ]:
# Get areas (Rows) for a specific SAFRA
subset_gdf = gdf[gdf['SAFRA'] == CropSeason]
subset_gdf = subset_gdf.drop(columns=['geometry'])

print('Rows:', subset_gdf.shape[0])
subset_gdf.head(2)

Rows: 4815


,ID_SIG,Local,AREA,COD_USINA,CODFAZ,BLOCO,TALHAO,VARIEDADE,TCH_ANT,POL,TAH,Unidade_So,Ordem,relevo,EST_Corte,VAR,SAFRA_real,soloGeral,SAFRA,usina
20677,X22230280001000100000027,Usina 1,19.811484,28,1,0,27,CT961007,112.79,0,0,RQ3,Neossolos,Ondulado e Suave Ondulado,1,CT961007,2122,RQ,2223,M1
20678,X22230280001000100000028,Usina 1,24.849480,28,1,0,28,CT961007,112.77,0,0,RQ3,Neossolos,Ondulado e Suave Ondulado,1,CT961007,2122,RQ,2223,M1


Merge datasets

In [ ]:
# Merge Agronomic information with monthly data based on the ID column
id_column = 'ID_SIG'

merged_gdf1 = subset_gdf.merge(CL_monthly, on=id_column, how='left')
merged_gdf2 = merged_gdf1.merge(HB_monthly, on=id_column, how='left')
merged_gdf3 = merged_gdf2.merge(S2_monthly, on=id_column, how='left')

print('Rows:',merged_gdf3.shape[0])
print('Columns:',merged_gdf3.shape[1],'\n')

merged_gdf3.head(2)

Rows: 4815
Columns: 308 



,ID_SIG,Local,AREA,COD_USINA,CODFAZ,BLOCO,TALHAO,VARIEDADE,TCH_ANT,POL,...,202202_rededge3,202203_rededge3,202112_swir1,202201_swir1,202202_swir1,202203_swir1,202112_swir2,202201_swir2,202202_swir2,202203_swir2
0,X22230280001000100000027,Usina 1,19.811484,28,1,0,27,CT961007,112.79,0,...,0.061071,0.061071,0.031640,0.030259,0.030259,0.030259,0.017622,0.016444,0.016444,0.016444
1,X22230280001000100000028,Usina 1,24.849480,28,1,0,28,CT961007,112.77,0,...,0.061533,0.061533,0.028754,0.026705,0.026705,0.026705,0.014574,0.012431,0.012431,0.012431


#### Exporting as table

In [ ]:
# Define the output path to save monthly BH data
data_all = os.path.join(out_path, f'06_all_data_monthly_safra_{CropSeason}.csv')
data_all

'/content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/06_merge_data_all/06_all_data_monthly_safra_2223.csv'

In [ ]:
# Save results as table
merged_gdf3.to_csv(data_all, encoding='utf-8', index=False)
print('Dowanloaded to: ', data_all)

Dowanloaded to:  /content/drive/MyDrive/Colab Notebooks/01_SugarcaneYieldPrediction/02_usinas_all/06_merge_data_all/06_all_data_monthly_safra_2223.csv
